# Recent  Approach : AthletePose3D - 3D Human Pose Estimation
Based on: Yeung et al. (2025) - AthletePose3D Dataset Paper

Implementation of methods from the paper:
- 2D Pose Detection (HRNet/ViTPose approach)
- 3D Pose Lifting (MotionAGFormer/TCPFormer)
- Kinematic Validation (Joint angles & velocities)
- Metrics: MPJPE, P-MPJPE, PDJ

Data  set : AthletePose3D: A Benchmark Dataset for 3D Human Pose Estimation and Kinematic Validation in Athletic Movements
Dataset Overview (Table 1 & 2 from Paper)

| Characteristic | Details |
|---------------|---------|
| **Total Frames** | 1.3 million frames |
| **Postures** | 165,000 individual postures |
| **Sports** | 3 categories, 12 motion types |
| **Athletes** | 8 athletes (amateur to professional) |
| **Keypoints** | 55/86 keypoints (varies by sport) |
| **Cameras** | 4-12 high-speed synchronized cameras |
| **Resolution** | 1920×1080 |

 Three Sport Categories :
![alt text](datset.jpg)


 1. Running (Lab Environment)
 2. Track & Field (Lab Environment)
 3. Figure Skating (Ice Rink)

we choose some running data  videos.

 # 1. Install Required Libraries 
What this does: Installs libraries mentioned in the paper

In [ ]:
import subprocess
import sys

print("="*70)
print(" INSTALLING LIBRARIES (Paper Technologies Only)")
print("="*70)

# Paper mentions these frameworks
packages = [
    'opencv-python',      # Video processing
    'numpy',              # Numerical computations
    'matplotlib',         # Visualization
    'scipy',              # Butterworth filter (Section 3.3)
    'torch',              # Deep learning framework
    'torchvision',        # Vision models
    'mediapipe',          # For demo (lightweight alternative)
    'pillow',             # Image processing
]

print("\nInstalling packages...")
for pkg in packages:
    print(f"  Installing {pkg}...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

print("\n✓ Installation complete!")
print("="*70)

# 2. Import Libraries and Setup
 What this does: Imports all required modules

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy.spatial.transform import Rotation
from mpl_toolkits.mplot3d import Axes3D
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("  AthletePose3D Implementation")
print("="*70)
print("\nPaper Information:")
print("  Title: AthletePose3D")
print("  Authors: Yeung et al. (2025)")
print("  Focus: 3D pose estimation for athletic movements")
print("  Dataset: 1.3M frames, 165K postures, 12 sports")
print("\nKey Technologies (from paper):")
print("  • 2D Models: HRNet, ViTPose, MogaNet (best: PDJ 95.7)")
print("  • 3D Models: MotionAGFormer, TCPFormer (best: MPJPE 98.26mm)")
print("  • Metrics: MPJPE, P-MPJPE, PDJ")
print("  • Kinematic: 4th-order Butterworth filter @ 8Hz")
print("  • Input: 81 frames per sequence")
print("  • Resolution: 1920×1080 @ 60/120 FPS")
print("="*70)

# 3. Load  Running Videos
What this does: Loads your 10-12 running videos

In [ ]:
VIDEO_FOLDER = "New_Paper"

print("\n" + "="*70)
print(" LOADING YOUR RUNNING VIDEOS")
print("="*70)

# Check if folder exists
if not os.path.exists(VIDEO_FOLDER):
    print(f"\n❌ ERROR: Folder '{VIDEO_FOLDER}' not found!")
    print("Please create the folder and add your running videos.")
    print("\nExpected structure:")
    print("  new_paper/")
    print("    ├── run1.mp4")
    print("    ├── run2.mp4")
    print("    └── ...")
else:
    # Find all video files
    video_files = []
    for ext in ['*.mp4', '*.avi', '*.mov', '*.MP4', '*.AVI', '*.MOV']:
        video_files.extend(Path(VIDEO_FOLDER).glob(ext))
    
    video_files = [str(f) for f in video_files]
    
    print(f"\n✓ Found {len(video_files)} videos:")
    for i, vf in enumerate(video_files, 1):
        # Get video info
        cap = cv2.VideoCapture(vf)
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        duration = frame_count / fps if fps > 0 else 0
        cap.release()
        
        print(f"  {i}. {os.path.basename(vf)}")
        print(f"     Resolution: {width}x{height}, FPS: {fps:.1f}, "
              f"Frames: {frame_count}, Duration: {duration:.1f}s")
    
    print(f"\nPaper specifications:")
    print(f"  • Running: 120 FPS (your videos may vary)")
    print(f"  • Resolution: 1920×1080 (paper standard)")
    print(f"  • Athletes: Inter-university level")

print("="*70)


# 4.Load and Process Single Video
 What this does: Loads one video for processing

In [ ]:
if len(video_files) > 0:
    # Select first video
    selected_video = video_files[0]
    print(f"\n📹 Processing: {os.path.basename(selected_video)}")
    
    # Load video frames
    cap = cv2.VideoCapture(selected_video)
    frames = []
    
    print("Loading frames...")
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"  Loaded {frame_count} frames...", end='\r')
    
    cap.release()
    frames = np.array(frames)
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    print(f"\n✓ Loaded {len(frames)} frames")
    print(f"  Shape: {frames.shape}")
    print(f"  FPS: {fps:.1f}")
    print(f"  Duration: {len(frames)/fps:.2f}s")
    
    # Display sample frames
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    sample_indices = [0, len(frames)//2, len(frames)-1]
    
    for ax, idx in zip(axes, sample_indices):
        ax.imshow(frames[idx])
        ax.set_title(f"Frame {idx}")
        ax.axis('off')
    
    plt.suptitle("Sample Frames from Your Running Video", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("\n⚠️ No videos found. Please add videos to new_paper/ folder")


# 5. Initialize 2D Pose Detector (Paper Models)
 What this does: Sets up 2D pose estimation
 Paper Section 3.2: HRNet, ViTPose, MogaNet

In [ ]:
print("\n" + "="*70)
print(" 2D POSE ESTIMATION MODELS")
print("="*70)

print("\nPaper's 2D Models (Section 3.2, Table 3):")
print("  1. HRNet (CVPR 2019) - CNN baseline - PDJ: 88.7")
print("  2. SwinPose (ICCV 2021) - Transformer - PDJ: 90.7")
print("  3. ViTPose (NeurIPS 2022) - Vision Transformer - PDJ: 95.0")
print("  4. UniFormer (ICLR 2022) - Transformer - PDJ: 95.2")
print("  5. MogaNet (ICLR 2024) - CNN (Best) - PDJ: 95.7 ⭐")

print("\nFor this demo, using MediaPipe Pose (lightweight alternative)")
print("  • Similar to HRNet architecture")
print("  • 33 keypoints detected")
print("  • Real-time performance")

import mediapipe as mp

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

pose_detector = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=2,  # Highest accuracy
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

print("\n✓ 2D Pose Detector initialized")
print("="*70)


# 6.Extract 2D Poses from Video
 What this does: Detects 2D keypoints in each frame
 Paper Section 3.2: 2D pose estimation with COCO format (17 keypoints)

In [ ]:
print("\n" + "="*70)
print(" EXTRACTING 2D POSES")
print("="*70)

def extract_2d_poses(frames, detector):
    """
    Extract 2D poses from video frames
    Paper: Uses models fine-tuned on COCO format (17 keypoints)
    """
    poses_2d = []
    confidences = []
    
    print(f"\nProcessing {len(frames)} frames...")
    
    for i, frame in enumerate(frames):
        if i % 20 == 0:
            print(f"  Frame {i}/{len(frames)}...", end='\r')
        
        # Convert to BGR for MediaPipe
        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        # Detect pose
        results = detector.process(frame_bgr)
        
        if results.pose_landmarks:
            # Extract landmarks (33 keypoints)
            landmarks = []
            conf_scores = []
            for lm in results.pose_landmarks.landmark:
                landmarks.append([lm.x, lm.y, lm.visibility])
                conf_scores.append(lm.visibility)
            
            poses_2d.append(np.array(landmarks))
            confidences.append(np.mean(conf_scores))
        else:
            # No pose detected
            poses_2d.append(np.zeros((33, 3)))
            confidences.append(0.0)
    
    print(f"\n✓ Extracted 2D poses for {len(poses_2d)} frames")
    return np.array(poses_2d), np.array(confidences)

# Extract poses
poses_2d, confidences = extract_2d_poses(frames, pose_detector)

print(f"\n2D Pose Extraction Results:")
print(f"  Shape: {poses_2d.shape}")
print(f"  Keypoints per frame: {poses_2d.shape[1]}")
print(f"  Detected frames: {np.sum(confidences > 0.5)}/{len(frames)}")
print(f"  Average confidence: {np.mean(confidences):.3f}")

print("\nPaper Format:")
print(f"  • Paper uses 17 COCO keypoints (we have 33)")
print(f"  • Input resolution: 384×288 (paper standard)")
print(f"  • Fine-tuned for 20 epochs")
print("="*70)


# isualize 2D Pose Detection
What this does: Shows detected skeleton on frames

In [ ]:
print("\n" + "="*70)
print(" VISUALIZING 2D POSE DETECTION")
print("="*70)

# MediaPipe connections
POSE_CONNECTIONS = [
    (11, 13), (13, 15),  # Left arm
    (12, 14), (14, 16),  # Right arm
    (11, 12),  # Shoulders
    (23, 25), (25, 27), (27, 29), (29, 31),  # Left leg
    (24, 26), (26, 28), (28, 30), (30, 32),  # Right leg
    (23, 24),  # Hips
    (11, 23), (12, 24)  # Torso
]

def draw_2d_pose(frame, pose_2d, connections):
    """Draw 2D pose on frame"""
    vis_frame = frame.copy()
    h, w = frame.shape[:2]
    
    # Draw keypoints
    for i, kp in enumerate(pose_2d):
        if kp[2] > 0.5:  # visibility threshold
            x, y = int(kp[0] * w), int(kp[1] * h)
            cv2.circle(vis_frame, (x, y), 6, (0, 255, 0), -1)
            cv2.circle(vis_frame, (x, y), 8, (255, 255, 255), 2)
    
    # Draw skeleton
    for connection in connections:
        pt1_idx, pt2_idx = connection
        if (pt1_idx < len(pose_2d) and pt2_idx < len(pose_2d) and 
            pose_2d[pt1_idx][2] > 0.5 and pose_2d[pt2_idx][2] > 0.5):
            pt1 = (int(pose_2d[pt1_idx][0] * w), int(pose_2d[pt1_idx][1] * h))
            pt2 = (int(pose_2d[pt2_idx][0] * w), int(pose_2d[pt2_idx][1] * h))
            cv2.line(vis_frame, pt1, pt2, (255, 0, 0), 3)
    
    return vis_frame

# Visualize
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
sample_indices = np.linspace(0, len(frames)-1, 8, dtype=int)

for ax, idx in zip(axes.flat, sample_indices):
    vis_frame = draw_2d_pose(frames[idx], poses_2d[idx], POSE_CONNECTIONS)
    ax.imshow(vis_frame)
    ax.set_title(f"Frame {idx} (Conf: {confidences[idx]:.2f})")
    ax.axis('off')

plt.suptitle("2D Pose Detection Results (Paper Section 5.1)", 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("✓ 2D pose visualization complete")
print("="*70)
